In [1]:
from API import API
api = API()

OK


In [3]:
import time
time.ctime(api.expiration)

'Tue Feb 28 13:56:39 2023'

In [ ]:
# TODO check whether token is expired
api.getPlaylists()

In [3]:
response = api.ClientCredentialsFlow()
response

OK


<Response [200]>

In [11]:
import json
json.loads(response.text)['expires_in']

3600

In [14]:
import time
time_og = time.time()
time_og

1677585282.0268934

In [15]:
time.ctime(time_og)

'Tue Feb 28 12:54:42 2023'

In [16]:
time.ctime(time_og + 3600)

'Tue Feb 28 13:54:42 2023'